In [1]:
import torch
import torch.nn as nn
layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size= (3,3))

ModuleNotFoundError: No module named 'torch'

In [2]:
layer(torch.randn(3,224,224)).shape

torch.Size([16, 222, 222])

In [6]:

table.measure_single_layer_flops(layer, input_size= (3,224,224))

1589.704704

In [11]:
size=0   #in kb
for param in layer.parameters():
    size += param.nelement() * param.element_size()
    print(param.element_size())
size = size / 1024

4
4


In [12]:
size

1.75

In [8]:
buffer_size

0